In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 81.8 MB/s 
     |████████████████████████████████| 6.6 MB 76.5 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 895 kB 97.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive')
PATH = './drive/MyDrive/datasets/'

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/groom_project2/answering
%pwd

/content/drive/MyDrive/groom_project2/answering


'/content/drive/MyDrive/groom_project2/answering'

In [6]:
def read_dev_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                temp_answer = []
                for answer in qa['answers']:
                    temp_answer.append(answer['text'])
                if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    contexts.append(context)
                    questions.append(question)
                    answers.append(temp_answer)

    return contexts, questions, answers

In [7]:
dev_contexts, dev_questions, dev_answers = read_dev_klue("/content/drive/MyDrive/groom_project2/answering/klue-mrc-v1.1_dev.json")

  0%|          | 0/5075 [00:00<?, ?it/s]

In [8]:
len(dev_answers)

4008

In [9]:
import csv

In [10]:
f = open('/content/drive/MyDrive/groom_project2/submissions/kobigbird1024_ep10_lr0.0001_774.csv','r',encoding='utf-8')
rdr = csv.reader(f)
prediction_answers = []
for line in rdr:
    prediction_answers.append(line[1])
f.close()

In [11]:
del prediction_answers[0]

In [12]:
prediction_answers[0]

''

In [13]:
def em_evalutate(prediction_answers, real_answers):
    total = len(prediction_answers)
    exact_match = 0
    for prediction_answer, real_answer in zip(prediction_answers, real_answers):
        if prediction_answer in real_answer:
            exact_match += 1
    
    return (exact_match/total) * 100

In [14]:
em_score = em_evalutate(prediction_answers, dev_answers)
em_score

67.19061876247505

In [15]:
def calc_distance(a, b):
    ''' 레벤슈타인 거리 계산하기 '''
    if a == b: return 0 # 같으면 0을 반환
    a_len = len(a) # a 길이
    b_len = len(b) # b 길이
    if a == "": return b_len
    if b == "": return a_len
    # 2차원 표 (a_len+1, b_len+1) 준비하기 --- (※1)
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1): # 0으로 초기화
        matrix[i] = [0 for j in range(b_len+1)]
    # 0일 때 초깃값을 설정
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j
    # 표 채우기 --- (※2)
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac == bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1,     # 문자 삽입
                matrix[i][j-1] + 1,     # 문자 제거
                matrix[i-1][j-1] + cost # 문자 변경
            ])
    return matrix[a_len][b_len]

In [16]:
calc_distance(prediction_answers[0], dev_answers[0][0])

16

In [17]:
prediction_answers[0], dev_answers[0][0]

('', '뉴 740Li 25주년 에디션')

In [18]:
len(dev_answers[0])

2

In [22]:
 from numpy import mean

In [23]:
score = []
for i in range(int(len(dev_answers))):
   s = calc_distance(prediction_answers[i], dev_answers[i][0])
   score.append(s)
mean(score)

2.6729041916167664